In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [53]:
customers_file= 'Customers_1.csv'
deposit_file= 'Deposit_1.csv'
loan_file= 'LoanData_1.csv'
savings_file= 'Savingdata_1.csv'

In [54]:
customers_file = pd.read_csv(customers_file)
deposit_file = pd.read_csv(deposit_file)
loan_file = pd.read_csv(loan_file)
savings_file = pd.read_csv(savings_file)

In [4]:
cust_per_branch = customers_file.groupby("Branch").count().to_dict()["Gender"]
cust_per_branch

{'AGENCE DE BENIN            ': 535,
 'AGENCE DE BOSTWANA          ': 559,
 'AGENCE DE KENYA ': 370,
 'AGENCE DE MADAGASCAR       ': 397,
 'AGENCE DE Mali    ': 294,
 'AGENCE DE NIGERIA            ': 39,
 'AGENCE DU GABO                ': 54,
 'AGENCE DU RWANDA': 1342,
 'AGENCE DU SENEGAL          ': 287,
 'AGENCE TANZANIE           ': 626}

In [5]:
gender_per_branch = pd.pivot_table(customers_file, values='Nationality', index=['Gender'], columns=['Branch'], aggfunc=np.count_nonzero)
gender_per_branch.to_dict()

{'AGENCE DE BENIN            ': {'C': 50, 'F': 152, 'M': 333},
 'AGENCE DE BOSTWANA          ': {'C': 54, 'F': 157, 'M': 348},
 'AGENCE DE KENYA ': {'C': 29, 'F': 116, 'M': 225},
 'AGENCE DE MADAGASCAR       ': {'C': 41, 'F': 111, 'M': 245},
 'AGENCE DE Mali    ': {'C': 32, 'F': 74, 'M': 188},
 'AGENCE DE NIGERIA            ': {'C': 3, 'F': 12, 'M': 24},
 'AGENCE DU GABO                ': {'C': 4, 'F': 15, 'M': 35},
 'AGENCE DU RWANDA': {'C': 138, 'F': 368, 'M': 836},
 'AGENCE DU SENEGAL          ': {'C': 22, 'F': 86, 'M': 179},
 'AGENCE TANZANIE           ': {'C': 76, 'F': 176, 'M': 374}}

In [89]:
customers_file["JoinDate"] = pd.to_datetime(customers_file["JoinDate"])
customers_file["JoinMonth"] = customers_file.JoinDate.dt.month_name().str.slice(stop = 3) + "-" + customers_file.JoinDate.dt.year.astype("str")
customers_file_sorted = customers_file.sort_values(by=['JoinDate'])
grouped_customers_by_month = customers_file_sorted.groupby("JoinMonth")

In [83]:
data = {}
for key, group in grouped_customers_by_month:
    
    group_data = group.groupby("Gender").count()["Branch"].to_dict()
    if "C" not in group_data:
        group_data["C"] = 0
    elif "M" not in group_data:
        group_data["M"] = 0
    elif "F" not in group_data:
        group_data["F"] = 0
        
    data[key] = group_data

## LOANS SECTION

In [8]:
loans_per_segment_count = loan_file.groupby("Segment").count().reset_index()[["Segment","Clined ID"]]
loans_per_segment_count.to_dict("list")

{'Segment': ['CORPORATE                     ',
  'MICRO ENTERPRISES             ',
  'NON BUSINESS ACTIVITIES-CORPOR',
  'RETAIL                        ',
  'SMEs-Small& medium enterp-CORP'],
 'Clined ID': [5, 20, 11, 2900, 225]}

In [9]:
loans_per_segment_sum = loan_file.groupby("Segment").sum().reset_index()[["Segment","Disbursement", "outstanding"]]
loans_per_segment_sum.to_dict("list")

{'Segment': ['CORPORATE                     ',
  'MICRO ENTERPRISES             ',
  'NON BUSINESS ACTIVITIES-CORPOR',
  'RETAIL                        ',
  'SMEs-Small& medium enterp-CORP'],
 'Disbursement': [1040938669.0,
  23412000.0,
  1447284999.0,
  9153729522.0,
  3007392940.0],
 'outstanding': [643616195.15,
  6806524.0,
  1052192762.0,
  6143680797.31,
  1972170272.0]}

In [10]:
loans_per_segment_sum = loan_file.sum(numeric_only=True).reset_index()
loans_per_segment_sum.to_dict("records")

[{'index': 'Clined ID', 0: 4997541.0},
 {'index': 'segcode', 0: 9299.0},
 {'index': 'Currency', 0: 3163.0},
 {'index': 'No.of.Installent', 0: 124154.0},
 {'index': 'Approved.Amount', 0: 14874204449.0},
 {'index': 'Disbursement', 0: 14672758130.0},
 {'index': 'Balance', 0: 201446319.0},
 {'index': 'paid_principal', 0: 4854291579.54},
 {'index': 'unpaid_principal', 0: 450210097.775},
 {'index': 'outstanding', 0: 9818466550.46}]

In [11]:
loan_file_copy = loan_file[:]

In [12]:
loan_file_copy["Approval_date"] = pd.to_datetime(loan_file["Approval.Date"])
loan_file_copy = loan_file_copy.sort_values(by=['Approval_date'])

C:\Users\INYANG\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [105]:
loan_file_copy["month"] = loan_file_copy["Approval_date"].dt.month_name().astype("str").str.slice(stop = 3) + "/" + loan_file_copy["Approval_date"].dt.year.astype("str")
loan_file_copy = loan_file_copy.sort_values("Approval_date")

loan_peformance_per_time = loan_file_copy.groupby(loan_file_copy.month, sort = False).sum()[["Approved.Amount", "Disbursement", "Balance", "paid_principal", "unpaid_principal", "outstanding"]]
loan_peformance_per_time.to_dict("index")

{'Jan/2004': {'Approved.Amount': 1070000.0,
  'Disbursement': 1070000.0,
  'Balance': 0.0,
  'paid_principal': 1069996.0,
  'unpaid_principal': 0.0,
  'outstanding': 4.0},
 'Apr/2004': {'Approved.Amount': 1500000.0,
  'Disbursement': 1500000.0,
  'Balance': 0.0,
  'paid_principal': 1499999.0,
  'unpaid_principal': 0.0,
  'outstanding': 1.0},
 'Aug/2004': {'Approved.Amount': 500000.0,
  'Disbursement': 500000.0,
  'Balance': 0.0,
  'paid_principal': 499999.0,
  'unpaid_principal': 0.0,
  'outstanding': 1.0},
 'Sep/2004': {'Approved.Amount': 100000.0,
  'Disbursement': 100000.0,
  'Balance': 0.0,
  'paid_principal': 99999.0,
  'unpaid_principal': 0.0,
  'outstanding': 1.0},
 'Oct/2004': {'Approved.Amount': 2500000.0,
  'Disbursement': 2500000.0,
  'Balance': 0.0,
  'paid_principal': 2499998.0,
  'unpaid_principal': 0.0,
  'outstanding': 2.0},
 'Nov/2004': {'Approved.Amount': 2000000.0,
  'Disbursement': 2000000.0,
  'Balance': 0.0,
  'paid_principal': 1999999.0,
  'unpaid_principal': 0.0

In [15]:
deposit_file["deposit_date"] = pd.to_datetime(deposit_file["dateofdeposit"])
deposit_file_copy = deposit_file[:]
deposit_file_copy = deposit_file.sort_values(by=['deposit_date'])

In [16]:
savings_file["saving_date"] = pd.to_datetime(savings_file["savingDate"])
# savings_file_copy = savings_file[:]
savings_file_copy = savings_file.sort_values(by=['savingDate'])
savings_file

,Client_No,D.O.B,Gender,Segment,Branch,savingDate,Type,SavingAmount,saving_date
0,1,5/1/1982,M,MICRO ENTERPRISES,AGENCE DE KENYA,2/10/2009,STUDENTS SAVINGS ACCOUNT,-12438,2009-02-10
1,2,5/1/1982,M,SMEs-Small& medium enterp-CORP,AGENCE DE KENYA,2/10/2009,STUDENTS SAVINGS ACCOUNT,-12438,2009-02-10
2,3,8/7/1984,M,RETAIL,AGENCE DE KENYA,2/11/2009,STUDENTS SAVINGS ACCOUNT,4268,2009-02-11
3,4,8/7/1984,M,SMEs-Small& medium enterp-CORP,AGENCE DE KENYA,2/11/2009,STUDENTS SAVINGS ACCOUNT,4268,2009-02-11
4,5,1/1/1985,M,SMEs-Small& medium enterp-CORP,AGENCE DE KENYA,2/13/2009,STUDENTS SAVINGS ACCOUNT,120,2009-02-13
5,6,1/1/1985,M,MICRO ENTERPRISES,AGENCE DE KENYA,2/13/2009,STUDENTS SAVINGS ACCOUNT,120,2009-02-13
6,7,9/29/1983,M,MICRO ENTERPRISES,AGENCE DE KENYA,2/20/2009,STUDENTS SAVINGS ACCOUNT,0,2009-02-20
7,8,9/29/1983,M,SMEs-Small& medium enterp-CORP,AGENCE DE KENYA,2/20/2009,STUDENTS SAVINGS ACCOUNT,0,2009-02-20
8,9,1/1/1983,M,MICRO ENTERPRISES,AGENCE DE KENYA,2/20/2009,STUDENTS SAVINGS ACCOUNT,119609,2009-02-20
9,10,1/1/1983,M,MICRO ENTERPRISES,AGENCE DE KENYA,2/20/2009,STUDENTS SAVINGS ACCOUNT,119609,2009-02-20


In [17]:
trimmed_saves = savings_file_copy[["Segment", "savingDate", "Branch", "SavingAmount"]].rename(columns = {"savingDate":"trans_date"})
trimmed_deps = deposit_file_copy[["Segment", "dateofdeposit", "Branch", "depositamount"]].rename(columns = {"dateofdeposit":"trans_date"})

In [18]:
full_deps_saves = pd.concat([trimmed_saves, trimmed_deps])

C:\Users\INYANG\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [19]:
full_deps_saves = full_deps_saves.fillna(0)

In [20]:
full_deps_saves["trans_date"] = pd.to_datetime(full_deps_saves["trans_date"])

In [21]:
full_deps_saves = full_deps_saves.sort_values(by = "trans_date")

In [22]:
full_deps_saves["date_str"] = full_deps_saves.trans_date.dt.month_name().str.slice(stop = 3) + "-" + full_deps_saves.trans_date.dt.year.astype("str") 

In [23]:
grouped_saves_deps = full_deps_saves.groupby(full_deps_saves["date_str"], sort = False).sum()

In [87]:
# full_deps_saves.to_dict("records")
grouped_saves_deps.to_dict("index")

{'May-1998': {'SavingAmount': 0.0, 'depositamount': 106140780.0},
 'Jun-1998': {'SavingAmount': 0.0, 'depositamount': 6000000.0},
 'Jul-1998': {'SavingAmount': 0.0, 'depositamount': 151600000.0},
 'Aug-1998': {'SavingAmount': 0.0, 'depositamount': 3519000.0},
 'Sep-1998': {'SavingAmount': 0.0, 'depositamount': 16982000.0},
 'Oct-1998': {'SavingAmount': 0.0, 'depositamount': 8918000.0},
 'Nov-1998': {'SavingAmount': 0.0, 'depositamount': 40435800.0},
 'Dec-1998': {'SavingAmount': 0.0, 'depositamount': 197627000.0},
 'Jan-1999': {'SavingAmount': 0.0, 'depositamount': 433550000.0},
 'Feb-1999': {'SavingAmount': 0.0, 'depositamount': 190618500.0},
 'Mar-1999': {'SavingAmount': 0.0, 'depositamount': 262774000.0},
 'Apr-1999': {'SavingAmount': 0.0, 'depositamount': 1095073700.0},
 'May-1999': {'SavingAmount': 0.0, 'depositamount': 1095690012.0},
 'Jun-1999': {'SavingAmount': 0.0, 'depositamount': 1994755000.0},
 'Jul-1999': {'SavingAmount': 0.0, 'depositamount': 2725628160.0},
 'Aug-1999': {